# Faster RCNN using TFOD 

## Load the annotations

In [1]:
import pandas as pd
import os

In [2]:
dataset_path = '../../dataset/self_driving_car_udacity/export'

In [3]:
annotations = pd.read_csv(dataset_path + '/_annotations.csv')

In [4]:
annotations.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,1478020642210920002_jpg.rf.0020e318772012f5778...,512,512,car,119,238,148,256
1,1478020642210920002_jpg.rf.0020e318772012f5778...,512,512,car,195,236,209,255
2,1478020642210920002_jpg.rf.0020e318772012f5778...,512,512,car,205,241,213,257
3,1478020642210920002_jpg.rf.0020e318772012f5778...,512,512,car,213,236,224,258
4,1478020642210920002_jpg.rf.0020e318772012f5778...,512,512,car,303,243,323,258


In [5]:
annotations = annotations.set_index('filename')

## Get the images

In [6]:
images = [f for f in os.listdir(dataset_path) if '.jpg' in f]

In [7]:
len(images)

15000

In [8]:
train_test_ratio = 0.9
train_test_limit = int(train_test_ratio * len(images))
train_image_names = images[:train_test_limit]
test_image_names = images[train_test_limit:]

In [9]:
print(len(train_image_names))
print(len(test_image_names))

13500
1500


## Create Label Map

In [10]:
labels = annotations['class'].unique()

In [11]:
labels

array(['car', 'pedestrian', 'trafficLight-GreenLeft',
       'trafficLight-Green', 'trafficLight-Red', 'trafficLight-RedLeft',
       'trafficLight', 'truck', 'biker', 'trafficLight-Yellow',
       'trafficLight-YellowLeft'], dtype=object)

In [12]:
def create_label_map():
    with open('label_map.pbtxt', 'w') as f:
        for i, l in enumerate(labels):
            f.write(
                'item {\n'+
                '    id: {}\n'.format(i+1)+
                '    name: \'{}\'\n'.format(l)+
                '}\n\n'
            )

create_label_map()

## Create TFRecords

In [13]:
from object_detection.utils import label_map_util
from object_detection.utils import dataset_util
from PIL import Image as pil_image
import io
import tensorflow as tf

In [14]:
label_map = label_map_util.load_labelmap('label_map.pbtxt')
label_map_dict = label_map_util.get_label_map_dict(label_map)

In [15]:
label_map_dict

{'car': 1,
 'pedestrian': 2,
 'trafficLight-GreenLeft': 3,
 'trafficLight-Green': 4,
 'trafficLight-Red': 5,
 'trafficLight-RedLeft': 6,
 'trafficLight': 7,
 'truck': 8,
 'biker': 9,
 'trafficLight-Yellow': 10,
 'trafficLight-YellowLeft': 11}

In [16]:
def create_tfrecords(image_names, record_name):
    writer = tf.io.TFRecordWriter(record_name)
    
    for image_name in image_names:
        if image_name not in annotations.index:
            continue
        image_path = os.path.join(dataset_path, image_name)
        with open(image_path, 'rb') as fid:
            encoded_jpg = fid.read()
        img_opened = pil_image.open(image_path)
        width, height = img_opened.size
        filename = image_path.encode('utf8')
        image_format = b'jpg'

        img_annot_details = annotations.loc[image_name]
        if type(img_annot_details['class']) is str:
            xmins = [img_annot_details['xmin']]
            ymins = [img_annot_details['ymin']]
            xmaxs = [img_annot_details['xmax']]
            ymaxs = [img_annot_details['ymax']]
            labels = [img_annot_details['class']]
        else:
            xmins = img_annot_details['xmin'].tolist()
            ymins = img_annot_details['ymin'].tolist()
            xmaxs = img_annot_details['xmax'].tolist()
            ymaxs = img_annot_details['ymax'].tolist()
            labels = img_annot_details['class'].tolist()
        classes = [label_map_dict[c] for c in labels]
        labels = [l.encode('utf8') for l in labels]

        example = tf.train.Example(features=tf.train.Features(feature={
            'image/height': dataset_util.int64_feature(height),
            'image/width': dataset_util.int64_feature(width),
            'image/filename': dataset_util.bytes_feature(filename),
            'image/source_id': dataset_util.bytes_feature(filename),
            'image/encoded': dataset_util.bytes_feature(encoded_jpg),
            'image/format': dataset_util.bytes_feature(image_format),
            'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
            'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
            'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
            'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
            'image/object/class/text': dataset_util.bytes_list_feature(labels),
            'image/object/class/label': dataset_util.int64_list_feature(classes),
        }))

        
        writer.write(example.SerializeToString())
        
    writer.close()

In [17]:
create_tfrecords(train_image_names, 'train.record')

In [18]:
create_tfrecords(test_image_names, 'test.record')

## Download a Pre-Trained Model


Model Zoo: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

## Configure Training Pipeline

```
mkdir my_training
cp faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/pipeline.config my_training/
```

Update the pipeline.config in my_training as per requirement

```
cp /home/sbjr/my_workspace/tfod-model-garden/research/object_detection/model_main_tf2.py .
```

## Training

```
python model_main_tf2.py --model_dir=faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/ --pipeline_config_path=my_training/pipeline.config
```